#### Function Calling
- OpenAI 모델이 사용자 코드 또는 외부 서비스와 상호 작용할 수 있도록 지원하는 기능
- LLM 모델은 데이터베이스 조회, 서버 API 호출, 파일저장 등 이런 작업 수행이 불가 => 실제 필요한 작업에 대해 함수 형태로 작성 후 호출 필요

In [2]:
from dotenv import load_dotenv, find_dotenv
# .env 파일 가져오기
load_dotenv(find_dotenv())

True

In [3]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-5-nano",
    input="2월 20일 삼성전자 주식 가격 알려줘"
)

print(response.output_text)

확인해 드리려면 연도(year)를 알려주셔야 정확한 가격을 드릴 수 있어요. 삼성전자 주식은 한국거래소 기준으로 005930.KS 티커를 사용합니다(또는 미국 ADR SSNLF).

참고로 Feb 20일은 해당 연도에 따라 거래일이 아닐 수도 있습니다(주말이거나 공휴일일 수 있음).

가져오는 방법 간단히 안내드려요
- Yahoo Finance: 005930.KS → Historical Data → 원하는 연도 Feb 20의 가격 확인
- Investing.com: Samsung Electronics Co Ltd (KS:005930) → Historical Data → Feb 20의 가격 확인
- Google Finance: 005930.KS Feb 20 [연도] 조회

원하시는 연도를 알려주시면 해당 연도 기준으로 가격 정보를 어떻게 확인하는지 더 구체적으로 도와드릴게요. 또한 한국거래소(KS) 기준 가격을 원하시는지, 미국 ADR(SSNLF) 기준 가격을 원하시는지도 알려주시면 좋습니다.


In [ ]:
# yahoo finance 호출 => 주식의 종목이 종목 코드로만 조회 가능
# !pip install yfinance

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 1.6/1.6 MB 50.3 MB/s  0:00:00
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15690 sha256=3e6440b2f6a22a13b7527351f0b4155e07c13726a829b0bf5a87c869a5877d7c
  Stored in directory: c:\users\louck\appdata\local\pip\cache\wheels\7e\62\f9\20d7dbb144b6f563edab8e3a7fda71d976870cd41972035cdd
Successfully built multitasking

   ---------------------------------------- 0/7 [peewee]
   ---------------------------------------- 0/7 [peewee]
   ---------------------------------------- 0/7 [peewee]
   ----------- ----------------


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import yfinance as yf

# 삼성전자
stock = yf.Ticker("005930.KS")
# data = stock.history(start="2026-02-20")
# 1d, 5d, 1mo, 3mo, 6mo..
data = stock.history(period="1d")
print(data)

                               Open      High       Low     Close   Volume  \
Date                                                                         
2026-02-23 00:00:00+09:00  194400.0  197600.0  194300.0  195300.0  9143450   

                           Dividends  Stock Splits  
Date                                                
2026-02-23 00:00:00+09:00        0.0           0.0  


- Open : 시작가
- High : 최고가
- Low : 최저가
- Close : 종가
- Volume : 거래량

In [ ]:
# 1) 스키마 정의 ( 형식 : JSON -> json 스키마)
# 2) 함수 작성
import json
import yfinance as yf
from openai import OpenAI
client = OpenAI()

tools = [
    {
        "type":"function",
        "name":"get_korea_stock_price",
        "description":"한국 주식 종목코드를 받아 현재 가격을 조회합니다.",
        "parameters":{
            "type":"object",
            "properties":{
                "code":{
                    "type":"string",
                    "description":"6자리 한국 주식 코드 (예: 005930)"
                }
            },
            "required":["code"]
        }
    }
]

def get_korea_stock_price(code):
    symbol = f"{code}.KS"
    stock = yf.Ticker(symbol)
    data = stock.history(period="1d")

    if data.empty:
        return json.dumps({"error":"종목 코드를 확인해 주세요"})

    latest = data.iloc[-1]
    return json.dumps({
        "code":code,
        "current_price":int(latest['Close']),
        "open":int(latest['Open']),
        "high":int(latest['High']),
        "low":int(latest['Low']),
        "volume":int(latest['Volume']),
    })

def run_korea_stock_agent(prompt):
    # prompt 값에 따라 tools 사용 여부 모델 판단
    response = client.responses.create(
        model="gpt-5-nano",
        tools=tools,   # ← tools 전달 필수
        input=[{"role":"user","content":prompt}]
    )

    # function 호출 결과만 수집
    function_calls = [item for item in response.output if item.type == "function_call"]

    if not function_calls:
        return response.output_text

    ## tool 호출이 여러번 할 수도 있음
    tool_outputs = []
    for call in function_calls:
        args = json.loads(call.arguments)
        result = get_korea_stock_price(**args)
        tool_outputs.append({
            "type":"function_call_output",
            "call_id":call.call_id,
            "output":result
        })

    # for 루프 밖에서 최종 응답 생성
    final_response = client.responses.create(
        model="gpt-5-nano",
        tools=tools,
        input=[
            {"role":"user","content":prompt},
            *response.output,
            *tool_outputs
        ]
    )
    return final_response.output_text

In [23]:
print(run_korea_stock_agent("삼성전자 현재 주가 알려줘"))

None


In [29]:
print(run_korea_stock_agent("005930 현재 주가 알려줘"))

죄송하지만 현재 실시간 주가를 바로 제공해 드리려면 실제 시세 조회가 필요합니다. 제 대화 세션에서 실시간 데이터에 직접 접속하는 기능은 지금 제공되지 않습니다.

원하시면 제가 인터넷에서 최신 가격을 찾아 알려드릴게요. 진행할까요? 허용해 주시면 바로 최신가를 확인해 드리겠습니다.

참고로 직접 확인하고 싶으시면 아래 페이지들에서 005930(삼성전자) 실시간 시세를 보실 수 있습니다.
- 네이버 금융: https://finance.naver.com/item/main.nhn?code=005930
- Yahoo Finance: https://finance.yahoo.com/quote/005930.KS
- Google 검색: "005930" 또는 "Samsung Electronics stock" (실시간 가격 표시 페이지로 연결)

원하시는 방법을 알려주세요. 또 실시간 외에 오늘의 종가나 변동률 같은 정보가 필요하신지도 말씀해 주세요.


In [37]:
from openai import OpenAI
import json
import requests

client = OpenAI()

tools = [
    {
        "type": "function",
        "name": "get_weather",
        "description": "도시/location 을 받아 현재 날씨(기온, 풍속 등)를 조회합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "예: 'Seoul', 또는 '서울' 같은 도시/지역명 입력",
                },
                "timezone": {
                    "type": "string",
                    "description": "예: 'Asia/Seoul', 또는 생략 가능",
                    "default": "auto"
                },
            },
            "required": ["location"],
        },
    },
]

def get_weather(location, timezone="auto"):
    # 위경도 찾기
    params = {"name": location, "count": 1, "language": "ko"}
    geo = requests.get("https://geocoding-api.open-meteo.com/v1/search", params=params, timeout=30).json()

    results = geo.get("results")
    if not results:
        return json.dumps({"error": f"{location} 위치를 찾지 못했습니다."})

    result = results[0]   # ← results[0]을 result에 저장
    lat, lon = result.get('latitude'), result.get('longitude')
    print(f"위,경도 {lat},{lon}")

    # 날씨 api 호출
    url = "https://api.open-meteo.com/v1/forecast"
    weather_params = {
        "latitude": lat,
        "longitude": lon,
        "current_weather": "true",
        "timezone": timezone
    }
    weather_result = requests.get(url, params=weather_params, timeout=30).json()
    current_weather = weather_result.get('current_weather')

    if not current_weather:
        return json.dumps({"error": "날씨 데이터를 가져오지 못했습니다."})

    return json.dumps({
        "location": location,
        "latitude": lat,
        "longitude": lon,
        "temperature_c": current_weather.get("temperature"),
        "windspeed_kmh": current_weather.get("windspeed"),
        "winddirection_deg": current_weather.get("winddirection"),
        "weathercode": current_weather.get("weathercode"),
        "time": current_weather.get("time"),
    })


def execute_tool(name, arguments):
    if name == "get_weather":
        return get_weather(**arguments)


def run_weather(prompt):
    input_list = [{"role": "user", "content": prompt}]

    response = client.responses.create(
        model="gpt-5-nano",
        tools=tools,
        input=input_list   # ← prompt를 input으로 전달
    )

    function_calls = [item for item in response.output if item.type == "function_call"]

    if not function_calls:
        return response.output_text

    tool_outputs = []

    for call in function_calls:
        args = json.loads(call.arguments)
        result = execute_tool(call.name, args)
        
        tool_outputs.append({
            "type": "function_call_output",
            "call_id": call.call_id,
            "output": result
        })

    # for 루프 밖에서 최종 응답 생성
    final_response = client.responses.create(
        model="gpt-5-nano",
        tools=tools,
        input=[
            {"role":"user","content":prompt},        # ← input_list 언패킹
            *response.output,
            *tool_outputs
        ]
    )
    return final_response.output_text

In [38]:
print(run_weather("Seoul 날씨 어때?"))

위,경도 37.566,126.9784
서울 현재 날씨 요약:
- 기온: 약 1.4°C
- 바람: 시속 약 9.6km/h
- 풍향: 약 283° (WNW 쪽)

시간 기준: 2026-02-23 13:00

오늘은 꽤 쌀쌀하니 두꺼운 옷 챙기세요. 더 자세한 예보가 필요하시면 알려 주세요.
